# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 10

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-096,1990.0,5,4,379,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.5,0.5,0.3,0.13,0.08,0.21,0.13,0.21,Matches
2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-062,1997.0,9,6,547,3,1,0,0,0,0,0.49,0.16,0.66,0.49,0.66,3.5,3.5,0.9,0.57,0.14,0.71,0.57,0.71,Matches
3,Che Adams\Che-Adams,eng ENG,FW,Southampton,24-143,1996.0,10,10,855,3,3,0,0,1,0,0.32,0.32,0.63,0.32,0.63,2.6,2.6,2.8,0.27,0.29,0.56,0.27,0.56,Matches
4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-070,1997.0,6,6,540,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.0,0.02,0.01,0.03,0.02,0.03,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,33-335,1987.0,2,2,180,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Tottenham,1,10,6,3,1,21,9,12,21,15.8,11.1,4.8,0.48,W W W W D,NaN,Son Heung-min - 9,Hugo Lloris,NaN
Liverpool,2,10,6,3,1,22,17,5,21,22.7,14.9,7.8,0.78,W W D W D,NaN,Mohamed Salah - 8,Alisson,NaN
Chelsea,3,10,5,4,1,22,10,12,19,15.8,9.8,6.0,0.60,D W W W D,NaN,Timo Werner - 4,Edouard Mendy,NaN
Leicester City,4,10,6,0,4,19,14,5,18,15.8,13.6,2.3,0.23,W W W L L,NaN,Jamie Vardy - 8,Kasper Schmeichel,NaN
West Ham,5,10,5,2,3,17,11,6,17,13.2,12.8,0.5,0.05,D L W W W,NaN,Jarrod Bowen - 4,Łukasz Fabiański,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-2,2,0.0,603,240,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,603,240,False,0.0,603,240,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.6,False,None,n,1,3,0.0,603,240,0,3339,9,51440,372,0.0,0.0,Özil,0
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-1,1,0.0,555,212,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,566,212,False,0.0,565,212,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,n,1,3,0.0,541,205,0,10224,4,18532,71,0.0,0.0,Sokratis,0
3,0,0,52,75.0,75.0,0,41270,NaN,,0,0,0,0,23.1,260,74,4.0,,0,2,0.0,0.4,0,David,0.0,7,0,11.7,285,95,False,72.0,223,85,364,Head injury - 75% chance of playing,2020-11-26T23:30:22.462494Z,55,0,0,NaN,0,,41270.jpg,1.2,0,0,Luiz Moreira Marinho,0.9,False,None,d,1,3,22.0,263,77,7,36815,114,77238,4073,0.0,1.3,David Luiz,0
4,1,1,117,100.0,100.0,3,54694,NaN,,0,0,-4,4,163.5,37,27,NaN,,0,3,2.3,2.8,2,Pierre-Emerick,2.3,12,2,55.4,38,23,False,139.6,117,42,896,,2020-10-08T09:00:12.305506Z,116,0,0,1.0,0,,54694.jpg,3.5,0,0,Aubameyang,10.4,False,None,a,1,3,251.0,31,15,35,458864,1429,2872244,66672,0.2,3.0,Aubameyang,2
5,0,0,0,100.0,100.0,0,58822,NaN,,0,0,-3,3,0.0,532,207,NaN,,0,2,0.0,0.5,0,Cédric,0.0,0,0,0.0,545,207,False,0.0,544,207,0,,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,,58822.jpg,0.0,0,0,Soares,0.2,False,None,a,1,3,0.0,508,197,0,3230,92,32469,296,0.0,0.0,Cédric,0


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Tottenham,1,10,6,3,1,21,9,12,21,15.8,11.1,4.8,0.48,W W W W D,NaN,Son Heung-min - 9,Hugo Lloris,NaN,1.580000,1.110000,0.329559
Liverpool,2,10,6,3,1,22,17,5,21,22.7,14.9,7.8,0.78,W W D W D,NaN,Mohamed Salah - 8,Alisson,NaN,2.270000,1.490000,0.225373
Chelsea,3,10,5,4,1,22,10,12,19,15.8,9.8,6.0,0.60,D W W W D,NaN,Timo Werner - 4,Edouard Mendy,NaN,1.580000,0.980000,0.375311
Leicester City,4,10,6,0,4,19,14,5,18,15.8,13.6,2.3,0.23,W W W L L,NaN,Jamie Vardy - 8,Kasper Schmeichel,NaN,1.580000,1.360000,0.256661
West Ham,5,10,5,2,3,17,11,6,17,13.2,12.8,0.5,0.05,D L W W W,NaN,Jarrod Bowen - 4,Łukasz Fabiański,NaN,1.320000,1.280000,0.278037
Southampton,6,10,5,2,3,19,16,3,17,10.6,12.7,-2.1,-0.21,W W W D L,NaN,Danny Ings - 5,Alex McCarthy,NaN,1.060000,1.270000,0.280832
Wolves,7,10,5,2,3,11,11,0,17,10.3,11.9,-1.6,-0.16,D W L D W,NaN,Raúl Jiménez - 4,Rui Patrício,NaN,1.030000,1.190000,0.304221
Everton,8,10,5,1,4,19,17,2,16,15.3,14.1,1.2,0.12,L L L W L,NaN,Dominic Calvert-Lewin - 10,Jordan Pickford,NaN,1.530000,1.410000,0.244143
Manchester Utd,9,9,5,1,3,16,16,0,16,13.3,11.2,2.1,0.23,D L W W W,NaN,Bruno Fernandes - 7,David de Gea,NaN,1.477778,1.244444,0.288101


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.6699604743083004

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,xPoints week 3,xG_week3,goals_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3,xPoints week 4,xG_week4,goals_week4,xGA_week4,cleansheet_week4,xA_week4,assists_week4,xPoints week 5,xG_week5,goals_week5,xGA_week5,cleansheet_week5,xA_week5,assists_week5,xPoints week 6,xG_week6,goals_week6,xGA_week6,cleansheet_week6,xA_week6,assists_week6,xPoints week 7,xG_week7,goals_week7,xGA_week7,cleansheet_week7,xA_week7,assists_week7,xPoints week 8,xG_week8,goals_week8,xGA_week8,cleansheet_week8,xA_week8,assists_week8,form 5,next1_xP,next5_xP,next10_xP,value,valuePoints metric,xG+xA,xG+xA per game
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,NaN,0,0,-2,2,0.0,601,239,NaN,NaN,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,601,239,False,0.0,601,239,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.6,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,NaN,0,0,-1,1,0.0,551,210,NaN,NaN,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,561,210,False,0.0,560,210,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,51,100.0,100.0,0,41270,NaN,NaN,0,0,0,0,9.8,316,105,4.0,NaN,0,2,0.5,0.5,0,David,0.0,5,0,9.7,286,95,False,66.6,219,89,319,NaN,2020-10-25T21:30:13.186040Z,55,0,0,NaN,0,NaN,41270.jpg,1.4,0,0,Luiz Moreira Marinho,0.9,False,...,1.199148,0.0,0.0,3.0,0.0,0.0,0.0,5.274923,0.0,0.0,0.2,0.0,0.0,0.0,3.586388,0.1,0.0,1.4,0.0,0.0,0.0,1.000000,0.0,0.0,1.2,0.0,0.0,0.0,NaN,NaN,NaN,0.3,0.0,NaN,NaN,NaN,NaN,NaN,1.8,0.0,NaN,NaN,2.293194,3.059255,2.583680,2.687033,0.457711,1.073427,0.1,0.02
4,1,1,115,100.0,100.0,3,54694,NaN,NaN,-1,1,-4,4,162.2,33,25,NaN,NaN,0,3,4.8,4.8,3,Pierre-Emerick,4.3,10,2,49.1,41,23,False,139.0,96,33,806,NaN,2020-10-08T09:00:12.305506Z,116,0,0,1.0,0,NaN,54694.jpg,3.7,0,0,Aubameyang,11.9,False,...,2.049787,0.0,0.0,3.0,0.0,0.0,0.0,2.818731,0.0,0.0,0.2,0.0,0.0,0.0,3.746597,0.0,0.0,1.4,0.0,0.5,0.0,4.101194,0.3,0.0,1.2,0.0,0.1,0.0,7.540818,0.9,1.0,0.3,1.0,0.1,0.0,2.765299,0.0,0.0,1.8,0.0,0.2,0.0,4.287199,3.855150,3.652955,3.769961,0.325363,1.108146,3.3,0.37
5,0,0,0,100.0,100.0,0,58822,NaN,NaN,0,0,-3,3,0.0,528,205,NaN,NaN,0,2,0.5,0.5,0,Cédric,0.0,0,0,0.0,540,205,False,0.0,539,205,0,NaN,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Tottenham,1,9,6,2,1,21,9,12,20,15.6,10.0,5.6,0.62,D W W W W,NaN,Son Heung-min - 9,Hugo Lloris,NaN,1.733333,1.111111,0.329193
Liverpool,2,9,6,2,1,21,16,5,20,22.3,12.8,9.4,1.05,D W W D W,NaN,Mohamed Salah - 8,Alisson,NaN,2.477778,1.422222,0.241177
Chelsea,3,9,5,3,1,22,10,12,18,14.8,9.7,5.1,0.57,D D W W W,NaN,Timo Werner - 4,Edouard Mendy,NaN,1.644444,1.077778,0.340351
Leicester City,4,9,6,0,3,18,12,6,18,15.0,11.7,3.3,0.36,L W W W L,NaN,Jamie Vardy - 8,Kasper Schmeichel,NaN,1.666667,1.300000,0.272532
Southampton,5,9,5,2,2,17,13,4,17,10.2,10.4,-0.2,-0.03,D W W W D,NaN,Danny Ings - 5,Alex McCarthy,NaN,1.133333,1.155556,0.314883


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff',"n'lundulu",'carlosvinicius']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == "n'lundulu":
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('lundulu')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'carlosvinicius':
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('carlos')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

603 is a new index. (xG)
603 is a new index. (xGA)
603 is a new index. (xA)


In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,assists_week2,xPoints week 3,xG_week3,goals_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3,xPoints week 4,xG_week4,goals_week4,xGA_week4,cleansheet_week4,xA_week4,assists_week4,xPoints week 5,xG_week5,goals_week5,xGA_week5,cleansheet_week5,xA_week5,assists_week5,xPoints week 6,xG_week6,goals_week6,xGA_week6,cleansheet_week6,xA_week6,assists_week6,xPoints week 7,xG_week7,goals_week7,xGA_week7,cleansheet_week7,xA_week7,assists_week7,xPoints week 8,xG_week8,goals_week8,xGA_week8,cleansheet_week8,xA_week8,assists_week8,xPoints week 9,xG_week9,goals_week9,xGA_week9,cleansheet_week9,xA_week9,assists_week9
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-2,2,0.0,603,240,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,603,240,False,0.0,603,240,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.6,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-1,1,0.0,555,212,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,566,212,False,0.0,565,212,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-21T10:30:18.546407Z,49,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,52,75.0,75.0,0,41270,NaN,,0,0,0,0,23.1,260,74,4.0,,0,2,0.0,0.4,0,David,0.0,7,0,11.7,285,95,False,72.0,223,85,364,Head injury - 75% chance of playing,2020-11-26T23:30:22.462494Z,55,0,0,NaN,0,,41270.jpg,1.2,0,0,Luiz Moreira Marinho,0.9,False,...,0.0,1.199148,0.0,0.0,3.0,0.0,0.0,0.0,5.274923,0.0,0.0,0.2,0.0,0.0,0.0,3.586388,0.1,0.0,1.4,0.0,0.0,0.0,1.000000,0.0,0.0,1.2,0.0,0.0,0.0,NaN,NaN,NaN,0.3,0.0,NaN,NaN,NaN,NaN,NaN,1.8,0.0,NaN,NaN,NaN,NaN,NaN,2.5,0.0,NaN,NaN
4,1,1,117,100.0,100.0,3,54694,NaN,,0,0,-4,4,163.5,37,27,NaN,,0,3,2.3,2.8,2,Pierre-Emerick,2.3,12,2,55.4,38,23,False,139.6,117,42,896,,2020-10-08T09:00:12.305506Z,116,0,0,1.0,0,,54694.jpg,3.5,0,0,Aubameyang,10.4,False,...,1.0,2.049787,0.0,0.0,3.0,0.0,0.0,0.0,2.818731,0.0,0.0,0.2,0.0,0.0,0.0,3.746597,0.0,0.0,1.4,0.0,0.5,0.0,4.101194,0.3,0.0,1.2,0.0,0.1,0.0,7.540818,0.9,1.0,0.3,1.0,0.1,0.0,2.765299,0.0,0.0,1.8,0.0,0.2,0.0,3.282085,0.3,0.0,2.5,1.0,-0.1,0.0
5,0,0,0,100.0,100.0,0,58822,NaN,,0,0,-3,3,0.0,532,207,NaN,,0,2,0.0,0.5,0,Cédric,0.0,0,0,0.0,545,207,False,0.0,544,207,0,,2020-09-23T09:00:14.881983Z,47,0,0,NaN,0,,58822.jpg,0.0,0,0,Soares,0.2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
388,Kane,10.000000,86,8.6,74.651067,7.465107
254,Salah,9.012346,73,8.1,64.191373,7.122604
302,Fernandes,9.024390,74,8.2,62.740409,6.952316
224,Vardy,9.041096,66,7.3,60.690356,6.712721
506,Wilson,9.054054,67,7.4,58.530356,6.464547
251,Mané,8.928571,50,5.6,55.582612,6.225252
37,Grealish,8.987342,71,7.9,55.445087,6.169242
390,Son,10.000000,87,8.7,61.475827,6.147583
202,Bamford,10.000000,62,6.2,60.580237,6.058024


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 280
Total xG: 267.0
goals per xG: 1.048689138576779
